# X.509 Certificates

With the cryptographic primitives you've learned, we can achieve confidentiality, integrity, authenticity, and even a bit of accountability. All of this, however, depends on our ability to distribute public keys reliably. If Mallory can trick Alice into accepting her public key instead of Bob's public key, she can subvert all of our security goals. We need a way to distribute public keys with trust. This is why X.509 was created.

I know. X.509 sounds like a Top Secret government program in a superhero movie. Unfortunately, it's is not nearly so exciting. X.509 is an internet standard for establishing trust. X.509 is ultimately just a protocol to distribute public keys reliably. Keep that in mind. When you go deep into the details of how X.509 works, it's easy to lose sight of the big picture. Try not to forget that X.509 is a way to give someone a public key in a way they can trust.

X.509 is a standard for **certificates** (sometimes shortened to "certs"). Certificates are files that contain the public key of some entity (e.g. Bob's public key.) A certificate's purpose is to establish to the reciever that it is a genuine document, so the receiver can trust that the public key it contains is authentic. We don't want Mallory to be able to forge a certificate with her own public key. 

Certificates can be downloaded in a format called PEM. I downloaded the cert for this website (jupyter.lawhack.net) using Firefox. The cert is included with this assignment in a file called "jupyter_lawhack_net.crt". Let's read the contents of that file into a string called "pem" and take a look at it.

In [ ]:
pem=open("jupyter_lawhack_net.crt").read()
print(pem)

Aside from some human readable ASCII text indicating the start and end of the certificate, this is not very informative. It looks like gibberish. The actual contents of the certificate are encoded in base 64. Base 64 is yet another numbering system used to encode data, like binary, decimal, and hexadecimal. Base 64 uses 64 symbols to encode each number. Rather than trying to decode the certificate data ourselves, we'll use a Python library to do the job for us. We will read the certificate into a variable named 'cert'.

In [ ]:
from cryptography import x509
from cryptography.hazmat.backends import default_backend

cert=x509.load_pem_x509_certificate(pem.encode('ASCII'), default_backend())

The variable "cert" now contains one of those fancy object datatypes we've encountered in the past. The library decoded the base 64 representation of the string and populated this object for us. We can use the scope operator to view the contents of the certificate object. Let's check what domain (website) the certificate belongs to and view its RSA public key.

In [ ]:
print(cert.subject)
publicKey=cert.public_key()
print("RSA public key",publicKey.public_numbers())

This certificate exists to prove that this public key belongs to jupyter.lawhack.edu. We will use this certificate as a running example as we continue to explore certificates.

## Certificate Authorities

**Certificate authorities** (usually abbreviated CAs) are entities (e.g. commercial enterprises or governments) responsible for creating and validating X.509 certificates. They act like notaries. We only trust a certificate if it has been signed by a trusted CA. CA's public keys are widely distributed. Typically, public keys for CAs are stored in the web browser or operating system. From those keys public keys, we derive all of our trust and security while browsing the web.

Let's see what CA issued our example certificate.

In [ ]:
print(cert.issuer)

Let's Encrypt is a nonprofit organization set up to act as a free CA. Their goal is to encourage website operators to encrypt their website traffic. I used their service to help secure this website. It's very convenient!

When our browser encounters a certificate, it looks for a CA's signature on the document. Let's examine the signature on our example cert. We'll check what algorithm was used to sign the cert and what the signature itself looks like.

In [ ]:
print(cert.signature_algorithm_oid)
print(cert.signature)

As you can see, the mess of bytes is the SHA256 hash digest of the certificate encrypted with the CA's (Let's Encrypt's) private key. This is a digital signature, exactly like the digital signatures we discussed in class.

Let's validate the signature using another Python library. Yes, the code we're about to look at is a little strange. Don't panic if you don't understand exactly how it all works. The code builds another object (fancy custom datatype) from the certificate string, then uses the scope operator to run a function on it. That much we've seen before in other examples, even if we don't quite know how it works.

This "try" and "except" business, on the other hand, is totally new. I'll do my best to explain how it works without going too deep. We *try* to execute all the code in the block beneath the **try** statement. If it fails for some reason, we bail out and execute the **except** code instead. In this case, we try to validate the certificate. If nothing goes wrong, we'll print "VALID!" If it fails, we bail out and print "INVALID!"

So let's check if this certificate is valid.

In [ ]:
from certvalidator import CertificateValidator

validator = CertificateValidator(pem.encode("ASCII"))

try: 
    validator.validate_usage(set(['digital_signature']))
    print("VALID!")
except:
    print("INVALID!")

All that weird code and hard work only to find out the certificate we use in this class is invalid? What gives?!

## Chain of Trust

x.509 certificates allow for **intermediate certificates**. A CA can mint and sign a certificate that allows someone else to sign certificates on the CA's behalf. We call the original CA the **root CA** and any others along the way **intermediate CAs**. This establishes a **chain of trust** from the **root CA** through the intermediate CAs to the certificates signed by the intermediate CAs. Our browser only contains the public keys for the root CAs. To validate a certificate signed by intermediate CAs, we need to validate each of the intermediate signing certificates first.

In this case, we need to include the Let's Encrypt certificate **and** their root CA's certificates in our validation of lawhack's certificate. This code loads two certificates stored in two files named "Let'sEncryptAuthorityX3.crt" and "DSTRootCAX3.crt". We then create a list of those intermediate certificates called "intermediates". Giving that list to the certificate validator allows it to check the entire chain of trust, validating our certificate.

In [ ]:
from certvalidator import CertificateValidator

interm1=open("Let'sEncryptAuthorityX3.crt",'rb').read()
interm2=open("DSTRootCAX3.crt",'rb').read()

intermediates=[interm1,interm2]
validator = CertificateValidator(pem.encode("ASCII"),intermediates)

try:
    validator.validate_usage(set(['digital_signature']))
    print("VALID!")
except:
    print("INVALID!")

By supplying the CertificateValidator library with the intermediate certificates, it can properly validate all of the signatures on the certificates in the chain of trust, letting us know that the lawhack certificate is, in fact, valid. Now we know that we can trust the public key it contains is authentic. Mission accomplished!

## Exercises

Below, you'll find the exercises for this notebook. They're not intended to be difficult from a programming perspective. Just follow the example code as closely as possible, and make sure you understand the X.509 concepts. If you encounter any unexpected programming problems, don't spend a lot of time struggling. Ask for help.

1) I have posted the code to load the certificate for the UT Austin Law School's website. What is the subject? What is the subject's public key? Who issued the certificate? What is its signature?

In [ ]:
pem=open("utw10007_utweb_utexas_edu.crt").read()

from cryptography import x509
from cryptography.hazmat.backends import default_backend

cert=x509.load_pem_x509_certificate(pem.encode('ASCII'), default_backend())

2) The intermediate certificates for the law school certificate are stored in files named "InCommonRSAServerCA.crt" and "USERTrustRSACertificationAuthority.crt". Load those intermediate certificates and validate the certificate.

In [ ]:
from certvalidator import CertificateValidator

pem=open("utw10007_utweb_utexas_edu.crt").read()

intermediates=[]

try:
    validator = CertificateValidator(pem.encode("ASCII"),intermediates)
    validator.validate_usage(set(['digital_signature']))
    print("VALID!")
except:
    print("INVALID!")

3) Use your browser to visit a website and view its certificate. Usually, this involves clicking on the lock icon to the left of the URL and clicking something like "view certificate" or "get more info". Tell me as much about the certificate as you can (e.g. who is the subject, who are the CAs and intermediate CAs and what cryptographic algorithms were used?)